In [1]:
import pickle
import numpy 
import torch
torch.set_printoptions(precision=10)

/mnt/data/anaconda3/envs/adat_pcdetv5/lib/python3.9/site-packages/tqdm-4.64.1-py3.9.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
path ="/netscratch/athreya/GT_Prototype_pkls/prototype_infos_fully_sup_nogtsmpl.pkl"
with open(path,"rb") as file:
    data=pickle.load(file)
data.keys()

dict_keys(['type', 'gt_boxes', 'gt_prototype'])

Compared to gt_auged which would have 16/27 boxes for bs2, here we have 9 boxes (example)

In [12]:
data['ens'][0].keys()

(8, 8)

In [ ]:

dim0 = data['ens'][0]['pooled_features_gt'].shape[0]
data['ens'][0]['gt_boxes'][...,-1]



In [ ]:
car_pool_list = []
ped_pool_list = []
cyc_pool_list = []
car_sh_list = []
ped_sh_list = []
cyc_sh_list = []
for ens in data['ens']:
    dim0 = ens['gt_classes'].shape[0]
    gt_val = torch.tensor(ens['pooled_features_gt']) # [N, 128, 6*6*6] ==>[N, 216, 128]
    gt_sh = torch.tensor([]) # ([30, 256])
    gt_classes = torch.tensor(ens['gt_boxes'])[...,-1]
    
    car_pool = gt_val[gt_classes==1]
    ped_pool = gt_val[gt_classes==2] 
    cyc_pool = gt_val[gt_classes==3]

    car_sh = gt_sh
    ped_sh = gt_sh
    cyc_sh = gt_sh
    
    car_pool_list.append(car_pool)
    ped_pool_list.append(ped_pool)
    cyc_pool_list.append(cyc_pool)

    car_sh_list.append(car_sh)
    ped_sh_list.append(ped_sh)
    cyc_sh_list.append(cyc_sh)

In [13]:


car_pool = torch.cat(car_pool_list)
cyc_pool = torch.cat(cyc_pool_list)
ped_pool = torch.cat(ped_pool_list)

car_sh = torch.cat(car_sh_list)
ped_sh = torch.cat(ped_sh_list)
cyc_sh = torch.cat(cyc_sh_list)

temp = {}
temp['Car'] = {}
temp['Ped'] = {}
temp['Cyc'] = {}

temp['Car']['mean'] = {}
temp['Ped']['mean'] = {}
temp['Cyc']['mean'] = {}

temp['Car']['mean']['sh'] = car_sh
temp['Ped']['mean']['sh'] = ped_sh
temp['Cyc']['mean']['sh'] = cyc_sh

temp['Car']['mean']['pool'] = car_pool
temp['Ped']['mean']['pool'] = ped_pool
temp['Cyc']['mean']['pool'] = cyc_pool

dict_keys(['type', 'gt_boxes', 'gt_prototype'])

In [ ]:


with open('/netscratch/athreya/GT_Prototype_pkls/tools/feature_bank_pooled_shared_0.01_gt_proto_nogtsmpl.pkl', 'wb') as f:
    pickle.dump(temp, f)


car_pool_prototype = car_pool.mean(dim=0)
ped_pool_prototype = ped_pool.mean(dim=0)
cyc_pool_prototype = cyc_pool.mean(dim=0)
car_sh_prototype = car_sh.mean(dim=0)
ped_sh_prototype = ped_sh.mean(dim=0)
cyc_sh_prototype = cyc_sh.mean(dim=0)

temp = {}
temp['Car'] = {}
temp['Ped'] = {}
temp['Cyc'] = {}

temp['Car']['mean'] = {}
temp['Ped']['mean'] = {}
temp['Cyc']['mean'] = {}

temp['Car']['mean']['sh'] = car_sh_prototype
temp['Ped']['mean']['sh'] = ped_sh_prototype
temp['Cyc']['mean']['sh'] = ped_sh_prototype

temp['Car']['mean']['pool'] = car_pool_prototype
temp['Ped']['mean']['pool'] = ped_pool_prototype
temp['Cyc']['mean']['pool'] = cyc_pool_prototype


In [ ]:
file_path = "/netscratch/athreya/GT_Prototype_pkls/prototype_infos_80pth_1_nogtsmpl.pkl"
with open(file_path, 'wb') as file:
    pickle.dump(temp, file)

print("Dictionary dumped to", file_path)